In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage
from langchain.chains import LLMChain

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()])

examples = [
    {
        "question": "What are three emojis for Titanic?",
        "answer": "🚢❤️💔",
    },
    {
        "question": "What are three emojis for Harry Potter?",
        "answer": "⚡🧙‍♂️📚",
    },
    {
        "question": "What are three emojis for the movie Avengers?",
        "answer": "🦸‍♂️🦸‍♀️💥",
    },
]

example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="""
    Previous conversations:
    {history}
    
    If the input is a movie name, reply with three emojis that represent the movie.
    If the user asks "What was the first movie I asked you about?", reply with the name of the first movie the user asked about, in plain text only. Do not use emojis in this case.
    
    Human: {movie}
    """,
    input_variables=["movie", "history"],
)

memory = ConversationBufferMemory(return_messages=True, memory_key="history")
chain = LLMChain(
    llm=chat,
    prompt=prompt,
    memory=memory
)




chain.run({"movie": "The godfather"})

AI: 🕴️🔫🍊

'AI: 🕴️🔫🍊'

In [22]:
chain.run({"movie": "Top Gun"})


AI: 🛩️🔥🕶️

'AI: 🛩️🔥🕶️'

In [24]:
for idx, entry in enumerate(memory.buffer):
    print(f"Entry {idx + 1}: content='{entry.content}' (Type: {type(entry)})")

def extract_first_movie_from_memory(memory):

    for entry in memory.buffer:
        if isinstance(entry, HumanMessage):  
            
            first_movie = entry.content.strip()  
            return first_movie  
    return None  

# Get the first movie from memory
first_movie = extract_first_movie_from_memory(memory)
print(f"\nFirst Movie from Memory: {first_movie}\n")


Entry 1: content='The godfather' (Type: <class 'langchain.schema.messages.HumanMessage'>)
Entry 2: content='AI: 🕴️🔫🍊' (Type: <class 'langchain.schema.messages.AIMessage'>)
Entry 3: content='Top Gun' (Type: <class 'langchain.schema.messages.HumanMessage'>)
Entry 4: content='AI: 🛩️🔥🕶️' (Type: <class 'langchain.schema.messages.AIMessage'>)

First Movie from Memory: The godfather

